In [1]:
import pandas as pd
import datetime


df=pd.read_csv('./csv/newdata0531.csv')
build_data=pd.read_csv('./csv/build.csv')

C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\3028537813.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('./csv/newdata0531.csv')
C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\3028537813.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  build_data=pd.read_csv('./csv/build.csv')


In [2]:
# 刪除重複資料
newdata = df.drop_duplicates(subset='編號', keep='first', inplace=False)
newbuild = build_data.drop_duplicates(subset='編號', keep='first', inplace=False)

新增index欄位 =原本index 

In [3]:
newdata['index'] = newdata.index
newdata['編號1'] = newdata['編號']

C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\1810874272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['index'] = newdata.index
C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\1810874272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdata['編號1'] = newdata['編號']


重新定義index(由'編號'定義)

In [4]:
newdata_1 = newdata.set_index('編號')
newbuild_1 = newbuild.set_index('編號')

資料合併

In [5]:
# 新增所需欄位
newdata_1['屋齡'] = None
newdata_1['建築完成日期'] = None
newdata_1['總層數'] = None

In [6]:
# 將有所需資料匯入原檔
newdata_1.update(newbuild_1)

新增單坪價格column

In [7]:
# 新增單坪價格欄位
newdata_1['單坪價格'] = newdata_1['總價元'] / newdata_1['建物坪數']

In [8]:
# 刪除'單價元平方公尺'
newdata_1 = newdata_1.drop('單價元平方公尺', axis=1)

刪除欄位

In [9]:
# 指定要刪除的欄位
dropcolumns = ['里', '交易座標橫坐標', '交易座標綜座標']

# 使用drop()函數刪除欄位
newdata_1.drop(dropcolumns, axis=1, inplace=True)

只保留 住宅大樓、華廈、公寓、套房、透天厝

In [10]:
valid_types = ['住宅大樓(11層含以上有電梯)', '華廈(10層含以下有電梯)', '公寓(5樓含以下無電梯)', '套房(1房1廳1衛)', '透天厝']
newdata_1 = newdata_1[newdata_1['建物型態'].isin(valid_types)]

只有台北市資料

In [11]:
newdata_1['縣市名稱'].value_counts()

縣市名稱
新北市    231438
桃園市    181414
台北市    105600
基隆市     23789
Name: count, dtype: int64

In [12]:
TP = newdata_1[newdata_1['縣市名稱'] == '台北市']

In [13]:
# 總價及單坪價格取到萬元
TP['總價(萬元)'] = TP['總價元'].apply(lambda x: round(x * 0.0001, 1))
TP['單坪價格(萬元)'] = TP['單坪價格'].apply(lambda x: round(x * 0.0001, 1))

C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\1970215278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TP['總價(萬元)'] = TP['總價元'].apply(lambda x: round(x * 0.0001, 1))
C:\Users\Sylvia\AppData\Local\Temp\ipykernel_848\1970215278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TP['單坪價格(萬元)'] = TP['單坪價格'].apply(lambda x: round(x * 0.0001, 1))


In [14]:
# 將DataFrame匯出為CSV檔案
TP.to_csv('./csv/TPdata0612.csv', index=False)